<a href="https://colab.research.google.com/github/Molten-Ice/Deep-Learning/blob/dev/GPT_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook I will be coding a GPT from scratch. 

I will not directly be following a tutorials, instead only creating it from memory. 

It's core component in Transformers, more precisely attention.

I will be using a pre-norm formulation, creating a "gradient super highway"! Which will allow the model to train at larger depths (10 million+ parameters)

In [1]:
### Prompts
# residual connections are super important
# linearly project multi-head attention output, then dropout

#feed forward linear(n, 4n), GeLU, linear(4n, n), dropout

#pre norm formulation, creates gradient super highway!
#layer norm before it goes into self-attention and feedforward

#add layer norms after block before final linear layer

#scaling up module
#dropout after softmax

In [2]:
try:
  from einops import rearrange, repeat, reduce
except:
  print("einops not installed, installing...")
  !pip install einops
  from einops import rearrange, repeat, reduce

einops not installed, installing...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 4.8 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn as nn

In [4]:
# hyperparameters
batch_size = 64 # num independent sequences processed in parallel 
block_size = 256 # what is the maximum context lengths?

max_iterations = 5000 # training iterations
eval_interval = 500 # how often to print out loss & accuracy
eval_iterations = 200 # how many batches to check during evaluation

learning_rate = 3e-4
dropout = 0.2

n = 8
n_embedding = 512 # each head has dim 64 (=512/8)
n_layer = 6
train_split = 0.9

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
# Importing data
file = "foundation.txt"
with open(file, 'r') as f:
  text = f.read()

print(f"Length of {file}: {len(text)} characters")
print(text[:250])

Length of foundation.txt: 1240543 characters
FOUNDATION 
ISAAC ASIMOV 

PART I 

THE PSYCHOHISTORIANS 

i. 

HARI SELDON-... bom In the 1 1,988th year of the Galactic Era; died 12,069. The dates are 
more commonly given In terms of the current Foundational Era as - 79 to the year 1 F.E. Born 
t


In [7]:
chars = sorted(list(set(text)))
n_chars = len(chars)
print(f"There are {n_chars} unique characters, namely: {''.join(chars)}")

There are 84 unique characters, namely: 
 !"#%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ\abcdefghijklmnopqrstuvwxyz—‘’”


In [8]:
ctoi = {ch:i for i, ch in enumerate(chars)} # characters to integers
itoc = {i:ch for i, ch in enumerate(chars)} # integers to character
encode = lambda s: [ctoi[ch] for ch in s]
decode = lambda l: ''.join([itoc[i] for i in l])
print(encode("Hello world!"))
print(decode(encode("Foo Bar!")))

encoded_text = encode(text)
print(len(encoded_text))

[34, 58, 65, 65, 68, 1, 76, 68, 71, 65, 57, 2]
Foo Bar!
1240543


In [9]:
n = int(len(encoded_text) * 0.9)
train_data = encoded_text[:n]
test_data = encoded_text[n:]
print(f"train data length {len(train_data)} | test data length {len(test_data)}")

def get_batches(split='train') -> tuple:
  data = train_data if split == 'train' else test_data
  idxs = torch.randint(len(encoded_text)-block_size, (batch_size, ))
  xb = torch.Tensor([encoded_text[i:i+block_size] for i in idxs]).long()
  yb = torch.Tensor([encoded_text[i+1:i+block_size+1] for i in idxs]).long()
  xb, yb = xb.to(device), yb.to(device)
  return xb, yb

xb, yb = get_batches()
xb.shape, yb.shape

train data length 1116488 | test data length 124055


(torch.Size([64, 256]), torch.Size([64, 256]))

In [10]:
@torch.no_grad()
def evaluate(model):
  model.eval()

  eval_iterations = 100 # temp

  splits = ['train', 'test']
  categories = ['loss', 'top1', 'top5']
  all = {s:{c: torch.zeros(eval_iterations) for c in categories} for s in splits}
  for split in splits:
    for i in range(eval_iterations):
      xb, yb = get_batches(split = split)
      logits, loss = model(xb, yb)
      all[split]['loss'][i] = loss.item()

      # top@1 accuracy
      top1_preds = torch.topk(logits, 1, dim = -1).indices.squeeze(dim=-1)
      all[split]['top1'][i] = (torch.sum(top1_preds == yb) / torch.numel(yb)).item()
      

      # top@5 accuracy
      top5_preds = torch.topk(logits, 5, dim = -1).indices
      y_stretched = repeat(yb, 'B T -> B T K', K = 5)
      all[split]['top5'][i] = (torch.sum(top5_preds == y_stretched) / torch.numel(yb)).item()
  
  output_str = ""
  for split in splits:

    loss = all[split]['loss'].mean().item()
    top1 = 100*all[split]['top1'].mean().item()
    top5 = 100*all[split]['top5'].mean().item()
    output_str+= f"{split} data -> loss:{loss:.4f}, top@1: {top1:.4f}%, top@5: {top5:.4f}% |"

  return output_str[:-2]

# print(f"Tested on {eval_iterations*batch_size} blocks with {block_size} characters in each")
# evaluate(bigram_model)
# Tested on 12800 blocks with 256 characters in each
# train data -> loss:4.8953, top@1: 0.0133%, top@5: 0.0609% |test data -> loss:4.8953, top@1: 0.0131%, top@5: 0.0608%
#1/n_chars = 0.0119

#prediction is as expected for a totally random system.

In [11]:
# To start with I will create a Bigram language model (i.e predict the next level ONLY using the previous letter)
class BigramLanguageModel(nn.Module):

  def __init__(self):
    super().__init__()

    # directly reads off logits for next character in table
    self.embedding = nn.Embedding(n_chars, n_chars)

  def forward(self, x: torch.Tensor, targets=None) -> torch.Tensor:

    logits = self.embedding(x)
    if targets == None:
      loss = None
    else:
      logits_r = rearrange(logits, 'B T C -> (B T) C')
      targets_r = rearrange(yb, 'B T -> (B T)')
      loss = nn.functional.cross_entropy(logits_r, targets_r)

    return logits, loss

bigram_model = BigramLanguageModel().to(device)
print(f'model parameters are on device: {next(bigram_model.parameters()).device}')
optimizer = torch.optim.Adam(params = bigram_model.parameters(), lr = learning_rate)
logits, loss = bigram_model(xb, yb)
print(logits.shape, loss)

model parameters are on device: cuda:0
torch.Size([64, 256, 84]) tensor(4.9692, device='cuda:0', grad_fn=<NllLossBackward0>)


In [12]:
### Training loop
for i in range(max_iterations):
  xb, yb = get_batches()

  logits, loss = bigram_model(xb, yb)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if i % eval_iterations == 0:
    print(evaluate(bigram_model))

train data -> loss:4.9611, top@1: 1.4244%, top@5: 5.0811% |test data -> loss:4.9623, top@1: 1.3861%, top@5: 5.0300%
train data -> loss:4.9028, top@1: 1.7072%, top@5: 5.4296% |test data -> loss:4.9039, top@1: 1.7076%, top@5: 5.3781%
train data -> loss:4.8488, top@1: 1.6911%, top@5: 5.3883% |test data -> loss:4.8482, top@1: 1.6902%, top@5: 5.3891%
train data -> loss:4.7926, top@1: 1.6917%, top@5: 7.7621% |test data -> loss:4.7920, top@1: 1.7105%, top@5: 7.7595%
train data -> loss:4.7407, top@1: 3.4842%, top@5: 8.3828% |test data -> loss:4.7414, top@1: 3.5187%, top@5: 8.3928%
train data -> loss:4.6927, top@1: 5.3740%, top@5: 8.8391% |test data -> loss:4.6937, top@1: 5.3738%, top@5: 8.8596%
train data -> loss:4.6466, top@1: 5.3845%, top@5: 9.9535% |test data -> loss:4.6467, top@1: 5.3824%, top@5: 9.9419%
train data -> loss:4.5949, top@1: 5.3909%, top@5: 13.3036% |test data -> loss:4.5950, top@1: 5.3923%, top@5: 13.2631%
train data -> loss:4.5540, top@1: 6.0031%, top@5: 16.0951% |test data 

In [ ]:
# train data -> loss:4.9611, top@1: 1.4244%, top@5: 5.0811% |test data -> loss:4.9623, top@1: 1.3861%, top@5: 5.0300%
# 5000 iterations later
# train data -> loss:4.0878, top@1: 17.0173%, top@5: 50.5164% |test data -> loss:4.0880, top@1: 16.9609%, top@5: 50.4433%